In [11]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import math
#import optuna
#import detectors
#import timm
from torchvision import transforms
# %matplotlib notebook
%matplotlib inline

In [3]:
# show the names all available GPU devices:
[torch.cuda.device(i) for i in range(torch.cuda.device_count())]
torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA H100 PCIe'

In [4]:
# Get the CIFAR10 dataset from 'torch':
from torchvision import datasets
from torchvision.transforms import ToTensor

# Download the training data from open datasets.
training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Create data loaders:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

100%|██████████| 170498071/170498071 [00:04<00:00, 40046252.12it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [7]:
model = timm.create_model("resnet18_cifar10", pretrained=True)

NameError: name 'timm' is not defined

In [ ]:
# calculate the model's accuracy on the test data:
model.eval()
correct = 0
total = 0

# CIFAR10 mean and standard deviation:
mean = [       0.4914,      0.4822,      0.4465    ]
std = [      0.2023,      0.1994,      0.201 ]
#mean = [       0.5,      0.5,      0.5    ]
#std = [      0.5,      0.5,      0.5 ]

normalize = transforms.Normalize(mean, std)

with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        images, labels = data
        # Normalize the images batch:
        images = normalize(images)
        outputs = model(images)
        predicted = outputs.argmax(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 10 == 0:
            print(f'For step {i} the accuracy is {100 * correct / total}%')

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

In [ ]:
def quant_tensor(tensor):
    quant_model_flatten = tensor
    if tensor.numel() >= 16:
        quant_model_flatten = quant_model_flatten.flatten()
        quant_model_flatten = quant_model_flatten.unfold(0, 16, 16).mean(-1)
        quant_model_flatten = quant_model_flatten.repeat_interleave(16)
        quant_model_flatten = quant_model_flatten.view(10, 512)

    return quant_model_flatten

def quantize_linear_layers(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            # Extract weights and biases
            weight = module.weight
            bias = module.bias

            # Quantize weights
            quantized_weight = quant_tensor(weight)

            # Replace original weights with quantized weights
            module.weight = nn.Parameter(quantized_weight, requires_grad=False)

            # Optional: Quantize biases if they exist
            if bias is not None:
                quantized_bias = quant_tensor(bias)
                module.bias = nn.Parameter(quantized_bias, requires_grad=False)
    return model

quant_model = quantize_linear_layers(model)

In [ ]:
# calculate the model's accuracy on the test data:
quant_model.eval()
correct = 0
total = 0

# CIFAR10 mean and standard deviation:
mean = [       0.4914,      0.4822,      0.4465    ]
std = [      0.2023,      0.1994,      0.201 ]
#mean = [       0.5,      0.5,      0.5    ]
#std = [      0.5,      0.5,      0.5 ]

normalize = transforms.Normalize(mean, std)

with torch.no_grad():
    for i, data in enumerate(test_dataloader):
        images, labels = data
        # Normalize the images batch:
        images = normalize(images)
        outputs = quant_model(images)
        predicted = outputs.argmax(dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if i % 10 == 0:
            print(f'For step {i} the accuracy is {100 * correct / total}%')

print(f'Accuracy of the quantized network on the 10000 test images: {100 * correct / total}%')

LoRA implementation and tests over MLP module and VGG pretrained

In [ ]:

vgg16_model = torchvision.models.vgg16(pretrained=True)

In [111]:
class LoRALayer(nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

class LinearWithLoRA(nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

def ReplaceLinearToLoRA(model, rank, alpha):
    for name, module in model.named_children():
        if isinstance(module, nn.Linear):
            setattr(model, name, LinearWithLoRA(module, rank=rank, alpha=alpha))
        else:
            ReplaceLinearToLoRA(module, rank=rank, alpha=alpha)

def FreeazeModel(model):
    for param in model.parameters():
        param.requires_grad = False
    
def UnfreezeLoRA(model):
    for child in model.children():
        if isinstance(child, LoRALayer):
            for param in child.parameters():
                param.requires_grad = True
        else:
            # Recursively freeze linear layers in children modules
            UnfreezeLoRA(child)

In [112]:
class MultilayerPerceptron(nn.Module):
    def __init__(self, num_features, 
        num_hidden_1, num_hidden_2, num_classes):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(num_features, num_hidden_1),
            nn.ReLU(),
            nn.Linear(num_hidden_1, num_hidden_2),
            nn.ReLU(),

            nn.Linear(num_hidden_2, num_classes)
        )

    def forward(self, x):
        x = self.layers(x)
        return x


model = MultilayerPerceptron(
    num_features=100,
    num_hidden_1=1000,
    num_hidden_2=1000, 
    num_classes=10
)

print(model)

MultilayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=100, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=1000, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1000, out_features=10, bias=True)
  )
)


In [109]:
ReplaceLinearToLoRA(model, 4, 8)
print(model)

MultilayerPerceptron(
  (layers): Sequential(
    (0): LinearWithLoRA(
      (linear): Linear(in_features=100, out_features=1000, bias=True)
      (lora): LoRALayer()
    )
    (1): ReLU()
    (2): LinearWithLoRA(
      (linear): Linear(in_features=1000, out_features=1000, bias=True)
      (lora): LoRALayer()
    )
    (3): ReLU()
    (4): LinearWithLoRA(
      (linear): Linear(in_features=1000, out_features=10, bias=True)
      (lora): LoRALayer()
    )
  )
)


In [ ]:
FreeazeModel(model)
UnfreezeLoRA(model)

for name, param in model.named_parameters():
    print(f"{name}: {param.requires_grad}")

In [ ]:
print(vgg16_model)

In [ ]:
ReplaceLinearToLoRA(vgg16_model, 4, 8)
print(vgg16_model)

In [ ]:
FreeazeModel(vgg16_model)
UnfreezeLoRA(vgg16_model)
for name, param in vgg16_model.named_parameters():
    print(f"{name}: {param.requires_grad}")